# Example usage

To use `meerkat_dl` in a project:

In [1]:
import mdl

print(mdl.__version__)

0.1.0


In [2]:
from mdl.net.layer import LinearLayer
import numpy as np

from mdl.net.activation import ReLU
from mdl.net.optimizers import GradientDescent
from mdl.net.loss import MeanSquaredLoss
from mdl.tensor import Tensor

In [3]:
np.random.seed(42)
num_samples = 1000
input_size = 5
output_size = 1
X = np.random.rand(num_samples, input_size).astype(np.float32)
true_weights = np.random.rand(input_size, output_size).astype(np.float32)
true_bias = np.random.rand(output_size).astype(np.float32)
y = np.dot(X, true_weights) + true_bias

In [4]:
X_tensor = Tensor(X)
y_tensor = Tensor(y)

In [5]:
model = LinearLayer(input_size, output_size)

In [6]:
num_epochs = 100
learning_rate = 0.01

optimizer = GradientDescent(model.aggregate_parameters(as_list=True), learning_rate=learning_rate)

In [7]:
activation_fn = ReLU()

In [8]:
for epoch in range(num_epochs):
    
    pred = model([X_tensor])
    activated_pred = activation_fn([pred])
    
    loss = MeanSquaredLoss(activated_pred, y_tensor)
    
    loss.backward()
    
    optimizer.step()
    
    optimizer.zero_grad()
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.data}')

ValueError: shapes (1,5) and (1000,5) not aligned: 5 (dim 1) != 1000 (dim 0)

In [7]:
import numpy as np

def gradient_checker(tensor, epsilon=1e-5):
    # Store the original tensor data
    original_data = tensor.data.copy()

    # Compute the gradients using automatic differentiation
    tensor.backward(np.array(1.0))
    autograd_grad = tensor.grad.copy()

    # Reset the tensor data to the original values
    tensor.data = original_data

    # Initialize an array to store the numerical gradients
    num_grad = np.zeros_like(original_data)

    # Iterate over each element in the tensor and compute the numerical gradient
    for idx in np.ndindex(original_data.shape):
        # Perturb the current element by epsilon
        tensor.data[idx] += epsilon

        # Compute the forward pass with the perturbed tensor
        perturbed_result = tensor.data.sum()  # Adjust as needed based on the operation

        # Reset the tensor data to the original values
        tensor.data = original_data

        # Perturb the current element by -epsilon
        tensor.data[idx] -= epsilon

        # Compute the forward pass with the perturbed tensor
        perturbed_result -= tensor.data.sum()  # Adjust as needed based on the operation

        # Reset the tensor data to the original values
        tensor.data = original_data

        # Compute the numerical gradient for the current element
        num_grad[idx] = perturbed_result / (2 * epsilon)

    # Compute the relative error between autograd and numerical gradients
    print(autograd_grad)
    print(num_grad)
    rel_error = np.linalg.norm(autograd_grad - num_grad) / np.linalg.norm(autograd_grad + num_grad)

    return rel_error


In [8]:
from mdl.tensor import Tensor

In [9]:
# Complex scenario with multiple operations and backward pass
tensor_a = Tensor(np.array([1, 2, 3]), requires_grad=True)
tensor_b = Tensor(np.array([4, 5, 6]), requires_grad=True)
tensor_c = Tensor(np.array([7, 8, 9]), requires_grad=True)

# Operations
result_add = tensor_a + tensor_b
result_mul = result_add * tensor_c
result_sum = result_mul.sum()

# Simulating backward pass
output_grad = np.array(1.0)
result_sum.backward(output_grad)

# Verify gradients using the gradient checker
error_a = gradient_checker(tensor_a)
error_b = gradient_checker(tensor_b)
error_c = gradient_checker(tensor_c)

# Check the errors
print(f"Gradient Checker Error for tensor_a: {error_a}")
print(f"Gradient Checker Error for tensor_b: {error_b}")
print(f"Gradient Checker Error for tensor_c: {error_c}")

deque([Tensor(172.0), Tensor([35. 56. 81.]), Tensor([5. 7. 9.]), Tensor([1. 2. 3.]), Tensor([4. 5. 6.]), Tensor([7. 8. 9.])])
Tensor(172.0)
Tensor([35. 56. 81.])
[Tensor(172.0)]
1.0
[0. 0. 0.]
[1. 1. 1.]
Tensor([5. 7. 9.])
[Tensor([35. 56. 81.])]
[1. 1. 1.]
[0. 0. 0.]
[7. 8. 9.]
Tensor([1. 2. 3.])
[Tensor([5. 7. 9.])]
[7. 8. 9.]
[0. 0. 0.]
[7. 8. 9.]
Tensor([4. 5. 6.])
[Tensor([5. 7. 9.])]
[7. 8. 9.]
[0. 0. 0.]
[7. 8. 9.]
Tensor([7. 8. 9.])
[Tensor([35. 56. 81.])]
[1. 1. 1.]
[0. 0. 0.]
[5. 7. 9.]


Exception: Shapes of gradient and Tensor need to match.